<a href="https://colab.research.google.com/github/gimenopea/MKTG6264-AIMLMktgAutomation/blob/main/Case_Study_3_Raw_Code_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

A casual dining chain in Columbus, OH that is known for its chicken sandwiches charges different prices at each of its restaurants and frequently changes the price of its sandwiches. In addition, each store has a coupon scheme whereby different value coupons are applicable at only certain restaurants (for example, someone would get a $1.25 coupon that is good only at the Italian Village location). ALmost all chicken sandwiches are sold using a coupon.

The resulting menu and marketing costs are high; and, while customers appreciate the quirkiness, there is anecdotal evidence that the unpredictability of prices drives customers away. Hence, the firm wants to investigate  whether it should standardize the cost of its sandwiches  at either the city-wide or the locational level (that is, make it uniform either within each restaurant or across all of the restaurants).

Specifically, your client wants to know :
a. whether it should unify the price of its chicken sandwiches across Columbus

b. whether it should unify the price at individual stores;

c. what the optimal price of the chicken sandwiches should be; and

d. whether it should continue to offer coupons, and, if so, what the optimal coupon should be.
This will involve a mixture of marketing theory and regression analysis.

Data
The client has given you data for January through April for all restaurants except one, which has February through April data. The data comprise the following variables:
- STORE : an indicator of the store

- MEDIAN_INC : the median income of the zipcode where the restaurant is located
- SOLD : how many chicken sandwiches were sold using the coupon
- PRICE : the menu price of a chicken sandwich at that store in that month
- DISCOUNT : the coupon discount given at that store for that month
- HAMBURGER : the price of a hamburger at the restaurant
- MENU_WA : the weighted average of all goods on the menu (weighted by volume sold).

You will work in groups. One or more members of your group will give a five-minute presentation on Wednesday, May 3. 

A final, actionable memo is due Friday, May 5, at 11:59 pm.

Only one member of your group needs to submit the meno online.
You will be graded on the actionable memo and the quality of your oral presentation.

### **Part 1. Data Prep**

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = '/content/drive/MyDrive/DATA_STORE/Chicken_Columbus.csv'

df = pd.read_csv(file_path)


In [3]:
#sample 10 random records
df.sample(10)

,STORE,MEDIAN_INC,SOLD,PRICE,DISCOUNT,HAMBURGER,MENU_WA
0,HIGH_ST,30179,27856,425,50,80,231.6
1,HIGH_ST,30179,29923,385,50,80,233.8
13,GER_VIL,51020,40489,525,95,125,327.0
22,BLOOR_ST,72099,52903,705,165,235,478.8
20,BLOOR_ST,72099,50144,590,125,220,449.8
21,BLOOR_ST,72099,51795,665,140,225,461.6
14,GER_VIL,51020,40731,490,90,130,330.6
7,ITAL_VIL,17876,36418,380,65,85,254.4
5,ITAL_VIL,17876,33388,385,60,85,247.2
9,COL_ST,33912,38483,405,70,95,269.4


In [4]:
# calculate averages for each store since some stores are missing monthly data (this way, we can only normalize everything as numbers per month)
averages = df.groupby('STORE')['SOLD'].mean().reset_index()
averages.rename(columns={'SOLD':'mean_sold'}, inplace=True)

averages['mean_discount'] = df.groupby('STORE')['DISCOUNT'].mean().reset_index()['DISCOUNT']


averages['mean_med_inc'] = df.groupby('STORE')['MEDIAN_INC'].mean().reset_index()['MEDIAN_INC']
averages['mean_price'] = df.groupby('STORE')['PRICE'].mean().reset_index()['PRICE']
averages['mean_hamburger_price'] = df.groupby('STORE')['HAMBURGER'].mean().reset_index()['HAMBURGER']
averages['mean_menu_wa'] = df.groupby('STORE')['MENU_WA'].mean().reset_index()['MENU_WA']

#format all the columns to 2 decimal places
averages = averages.round(2)

averages

,STORE,mean_sold,mean_discount,mean_med_inc,mean_price,mean_hamburger_price,mean_menu_wa
0,BLOOR_ST,51364.50,138.75,72099.0,645.00,221.25,449.75
1,BROAD_ST,43462.00,122.50,41290.0,592.50,150.00,370.00
2,COL_ST,38545.33,66.67,33912.0,393.33,98.33,270.73
3,GER_VIL,40850.25,81.25,51020.0,453.75,118.75,309.60
4,HIGH_ST,29601.75,51.25,30179.0,402.50,80.00,235.05
5,ITAL_VIL,34172.50,60.00,17876.0,383.75,83.75,247.90


### **Part 2. Create regression model for the given dataset**

In [5]:
import statsmodels.api as sm

# define the target and predictor variables
y = df['SOLD']
X = df[['MEDIAN_INC', 'PRICE', 'DISCOUNT', 'HAMBURGER', 'MENU_WA']]

# add constant to predictor variables
X = sm.add_constant(X)

# fit and summarize the OLS (ordinary least squares) model


model = sm.OLS(y, X).fit()

predicted = model.fittedvalues
residuals = model.resid

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   SOLD   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     80.92
Date:                Mon, 17 Apr 2023   Prob (F-statistic):           3.04e-11
Time:                        12:29:28   Log-Likelihood:                -200.04
No. Observations:                  23   AIC:                             412.1
Df Residuals:                      17   BIC:                             418.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        4.91e+04   7019.136      6.995      0.0

In [6]:
# determine the regression equation
target_var = 'SOLD'
predictor_vars = ['MEDIAN_INC', 'PRICE', 'DISCOUNT', 'HAMBURGER', 'MENU_WA']

X = df[predictor_vars]
y = df[target_var]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

coefs = model.params
equation = f'SOLD = {coefs[0]:.2f}'
for i in range(1,len(predictor_vars)+1):
    coef_label = coefs.index[i]
    coef_val = coefs[i]
    equation += f' + {coef_val:.2f} * {coef_label}'

print(equation)

SOLD = 49099.86 + 0.13 * MEDIAN_INC + -83.86 * PRICE + 399.66 * DISCOUNT + 127.35 * HAMBURGER + -80.41 * MENU_WA


### **Part 3. Explaining regression output:**

We can see the impact of each variable on the model based on their coefficients. 

- `MEDIAN_INC`: For a one-unit increase in median income, the predicted value of SOLD will increase by 0.1284 units, holding other variables.

- `PRICE`: For a one-unit increase in price, the predicted value of SOLD will decrease by 83.8607 units, holding other variables constant.

- `DISCOUNT`: For a one-unit increase in discount, the predicted value of SOLD will increase by 399.6590 units, holding  other variables constant.

- `HAMBURGER`: For a one-unit increase in hamburger sales, the predicted value of SOLD will increase by 127.3455 units, holding other variables constant.

- `MENU_WA`: For a one-unit increase in menu advertisements, the predicted value of SOLD will decrease by 80.41 units, holding other variables constant.

Therefore based on the given regression coefficients, we can conclude that the most impactful feature in predicting SOLD is `DISCOUNT`, followed by the price of a `HAMBURGER` and `MEDIAN_INC`, which is median income on which the restaurant is located. On the other hand, PRICE has a negative impact on SOLD, while MENU_WA has relatively less impact.


### **Part 4. Initial Analysis**

Now that we know the regression coefficients and know that it has a relatively large R2 value, we can focus our strategy on key areas:

- Start by modifying `DISCOUNT` levels for each of the stores per month, keeping everything else constant. Determing the sensitivity for demand given the level of discount.

- Once sales volume hits it's peak or we see diminishing returns for a specified discount for a store, we can adjust hamburger prices for the same discount until demand falls and we see diminishing returns.

- For optimal performance, since we know that median_income is a correlated variable to the amount of chicken sandwiches sold, we can raise discounts and hamburger prices in higher increments in stores with higher median incomes than stores with lower median incomes

- Since there is a negative correlation with the volume of chicken sandwiches sold for its price, we should unify the price of chicken sandwiches across all stores. 

- Since there is a negative correlation with the weighted average of all goods on the menu, we should simplify them across all stores. We can start by normalizing the price based on the mean sales for a particular store.

- To get the optimal price of the chicken sandwich, we can use the regression line equation and the mean value for each of the stores

Step 1: 

`MEAN_SOLD_ALL_STORES = 49099.86 + 0.13 * MEAN_MEDIAN_INC - 83.86 * OPTIMAL_PRICE + 399.66 * MEAN_DISCOUNT + 127.35 * MEAN_HAMBURGER - 80.41 * MEAN_MENU_WA`

Step 2:

`83.86 * OPTIMAL_PRICE = 49099.86 + 0.13 * MEAN_MEDIAN_INC + 399.66 * MEAN_DISCOUNT + 127.35 * MEAN_HAMBURGER - 80.41 * MEAN_MENU_WA - MEAN_SOLD`

Step 3:


`OPTIMAL_PRICE = (48675.28 + 0.13 * MEAN_MEDIAN_INC + 399.66 * MEAN_DISCOUNT + 127.35 * MEAN_HAMBURGER - 80.41 * MEAN_MENU_WA) / 83.86`

In [7]:
# here are the monthly averages per each store

averages.set_index('STORE', inplace=True)
averages

,mean_sold,mean_discount,mean_med_inc,mean_price,mean_hamburger_price,mean_menu_wa
STORE,,,,,,
BLOOR_ST,51364.50,138.75,72099.0,645.00,221.25,449.75
BROAD_ST,43462.00,122.50,41290.0,592.50,150.00,370.00
COL_ST,38545.33,66.67,33912.0,393.33,98.33,270.73
GER_VIL,40850.25,81.25,51020.0,453.75,118.75,309.60
HIGH_ST,29601.75,51.25,30179.0,402.50,80.00,235.05
ITAL_VIL,34172.50,60.00,17876.0,383.75,83.75,247.90


In [8]:
# here are all the coefficients

const_coeff = coefs[0]
med_inc_coeff = coefs[1]
price_coeff = coefs[2]
discount_coeff = coefs[3]
hamburger_coeff = coefs[4]
menu_wa_coeff = coefs[5]

#print all coefficients
print('Constant Coefficient: ', const_coeff)
print('Median Income Coefficient: ', med_inc_coeff)
print('Price Coefficient: ', price_coeff)
print('Discount Coefficient: ', discount_coeff)
print('Hamburger Coefficient: ', hamburger_coeff)
print('Menu WA Coefficient: ', menu_wa_coeff)

Constant Coefficient:  49099.85578606592
Median Income Coefficient:  0.1283736177209982
Price Coefficient:  -83.860691155122
Discount Coefficient:  399.65896696697484
Hamburger Coefficient:  127.34548581335082
Menu WA Coefficient:  -80.4099909051804


In [9]:
# calculating the optimal price of a chicken sandwich and optimal discounts (coupon? for each store with our regression model

optimal_prices = {}
optimal_discounts = {}
for i in range(0, len(averages)):
    #save index name to variable store
    store = averages.index[i]
    med_inc = averages.iloc[i,2]
    price = averages.iloc[i,3]
    discount = averages.iloc[i,1]
    hamburger = averages.iloc[i,4]
    menu_wa = averages.iloc[i,5]
    mean_sold= averages.iloc[i,0]
    optimal_prices[store] = abs((const_coeff - mean_sold +
                              med_inc_coeff * med_inc + 
                              discount_coeff * discount + 
                              hamburger_coeff * hamburger + 
                              menu_wa_coeff * menu_wa)/price_coeff)
    optimal_discounts[store] = abs((const_coeff - mean_sold +
                              med_inc_coeff * med_inc + 
                              price_coeff * price + 
                              hamburger_coeff * hamburger + 
                              menu_wa_coeff * menu_wa)/discount_coeff)

#building a final table with averages per each store and all the corresponding optimal discounts and chicken sandwich prices
optimal_discounts = pd.DataFrame.from_dict(optimal_discounts, orient='index').reset_index()
optimal_discounts.rename(columns={'index':'store', 0:'optimal_discount'}, inplace=True)

optimal_prices = pd.DataFrame.from_dict(optimal_prices,orient='index').reset_index()
optimal_prices.rename(columns={'index':'store', 0:'optimal_price'}, inplace=True)

optimal_prices = pd.merge(optimal_prices, optimal_discounts, on='store')
optimal_prices.set_index('store', inplace=True)

averages = pd.merge(averages, optimal_prices,left_index=True, right_index=True)

In [10]:
averages

,mean_sold,mean_discount,mean_med_inc,mean_price,mean_hamburger_price,mean_menu_wa,optimal_price,optimal_discount
STORE,,,,,,,,
BLOOR_ST,51364.50,138.75,72099.0,645.00,221.25,449.75,649.344067,137.838482
BROAD_ST,43462.00,122.50,41290.0,592.50,150.00,370.00,587.244768,123.602709
COL_ST,38545.33,66.67,33912.0,393.33,98.33,270.73,385.230310,68.369563
GER_VIL,40850.25,81.25,51020.0,453.75,118.75,309.60,447.156607,82.633496
HIGH_ST,29601.75,51.25,30179.0,402.50,80.00,235.05,419.053138,47.776645
ITAL_VIL,34172.50,60.00,17876.0,383.75,83.75,247.90,380.789233,60.621260


### Construct pricing ladder given initial optimal_pricing and optimal_discount from the regression model

In [67]:
#get min and max ranges of chicken sandwiches sold for all stores to set our initial pricing model

min_sold = df['SOLD'].describe()['min']
max_sold = df['SOLD'].describe()['max'] + 50000 #adding 50k here for future sales

print(f'min sandwiches sold {min_sold}')
print(f'max sandwiches sold {max_sold}')


min sandwiches sold 27856.0
max sandwiches sold 102903.0


In [89]:
def calculate_optimal_discount(sold, median_inc, menu_wa, hamburger, price):
    discount = abs((sold - const_coeff - med_inc_coeff * median_inc + price_coeff * price - hamburger_coeff * hamburger + menu_wa_coeff * menu_wa) / discount_coeff)
    return round(discount)

def calculate_optimal_price(sold, median_inc, menu_wa, hamburger, discount):
    price = abs((sold - const_coeff - med_inc_coeff * median_inc + discount_coeff * discount - hamburger_coeff * hamburger + menu_wa_coeff * menu_wa) / price_coeff)
    return round(price)

def pricing_df_store(store, med_inc, price, discount, hamburger, menu_wa, mean_sold, min_sold, max_sold):
  input_median_inc = med_inc
  input_menu_wa = menu_wa
  input_hamburger = hamburger
  input_price = price

  sold_min = int(min_sold)
  sold_max = int(max_sold)
  sold_step = 2500

  discounts = []
  hamburgers = []
  month_count = 1
  for sold in range(sold_min, sold_max+1, sold_step):
      discount = calculate_optimal_discount(sold, input_median_inc, input_menu_wa, input_hamburger, input_price)
      hamburger = calculate_optimal_price(sold, input_median_inc, input_menu_wa, input_hamburger, discount)
      discounts.append({"STORE": store, "PERIOD": month_count,"SOLD": sold, "DISCOUNT": discount}) 
      hamburgers.append({"STORE": store, "PERIOD": month_count,"SOLD": sold, "HAMBURGER_PRICE": hamburger})   
      month_count += 1
    
  df_discounts = pd.DataFrame(discounts)
  df_hamburgers = pd.DataFrame(hamburgers)
  return df_discounts, df_hamburgers
 

In [90]:
pricing_ladder_discounts = pd.DataFrame()
pricing_ladder_hamburgers = pd.DataFrame()

for i in range(0, len(averages)):    
    store = averages.index[i]
    med_inc = averages.iloc[i,2]
    price = averages.iloc[i,3]
    discount = averages.iloc[i,1]
    hamburger = averages.iloc[i,4]
    menu_wa = averages.iloc[i,5]
    mean_sold= averages.iloc[i,0]

    df_discounts, df_hamburgers = pricing_df_store(store, med_inc, price, discount, hamburger, menu_wa, mean_sold, min_sold, max_sold)
   

    pricing_ladder_discounts = pd.concat([pricing_ladder_discounts,df_discounts], axis=1)
    pricing_ladder_hamburgers = pd.concat([pricing_ladder_hamburgers,df_hamburgers],axis=1)

    

In [91]:
pricing_ladder_discounts

,STORE,PERIOD,SOLD,DISCOUNT,STORE,PERIOD,SOLD,DISCOUNT,STORE,PERIOD,...,SOLD,DISCOUNT,STORE,PERIOD,SOLD,DISCOUNT,STORE,PERIOD,SOLD,DISCOUNT
0,BLOOR_ST,1,27856,373,BROAD_ST,1,27856,313,COL_ST,1,...,27856,265,HIGH_ST,1,27856,220,ITAL_VIL,1,27856,216
1,BLOOR_ST,2,30356,366,BROAD_ST,2,30356,307,COL_ST,2,...,30356,259,HIGH_ST,2,30356,214,ITAL_VIL,2,30356,210
2,BLOOR_ST,3,32856,360,BROAD_ST,3,32856,300,COL_ST,3,...,32856,252,HIGH_ST,3,32856,208,ITAL_VIL,3,32856,203
3,BLOOR_ST,4,35356,354,BROAD_ST,4,35356,294,COL_ST,4,...,35356,246,HIGH_ST,4,35356,201,ITAL_VIL,4,35356,197
4,BLOOR_ST,5,37856,348,BROAD_ST,5,37856,288,COL_ST,5,...,37856,240,HIGH_ST,5,37856,195,ITAL_VIL,5,37856,191
5,BLOOR_ST,6,40356,341,BROAD_ST,6,40356,282,COL_ST,6,...,40356,234,HIGH_ST,6,40356,189,ITAL_VIL,6,40356,185
6,BLOOR_ST,7,42856,335,BROAD_ST,7,42856,275,COL_ST,7,...,42856,227,HIGH_ST,7,42856,183,ITAL_VIL,7,42856,178
7,BLOOR_ST,8,45356,329,BROAD_ST,8,45356,269,COL_ST,8,...,45356,221,HIGH_ST,8,45356,176,ITAL_VIL,8,45356,172
8,BLOOR_ST,9,47856,323,BROAD_ST,9,47856,263,COL_ST,9,...,47856,215,HIGH_ST,9,47856,170,ITAL_VIL,9,47856,166
9,BLOOR_ST,10,50356,316,BROAD_ST,10,50356,257,COL_ST,10,...,50356,209,HIGH_ST,10,50356,164,ITAL_VIL,10,50356,160


### Ladder strategy for discounts:

1. We pick a starting period based on our last monthly chicken sandwich sales. For example, for store: Bloor St. we start on period 10 and apply a discount of 310. If we reach the next threshold in sales of 55356 (period 11), then we increase the discount until the sales plateau.

2. We follow the same strategy for each store, moving up in period or downgrading to the prior period based on demand sensitivity

3. Before adjusting discount however, we can adjust hamburger prices first using a similar strategy and see if sales improve keeping discount and everything else constant

4. If sales pleateau after adjusting hamburger prices, we can proceed to adjusting discount prices again

In [92]:
pricing_ladder_hamburgers


,STORE,PERIOD,SOLD,HAMBURGER_PRICE,STORE,PERIOD,SOLD,HAMBURGER_PRICE,STORE,PERIOD,...,SOLD,HAMBURGER_PRICE,STORE,PERIOD,SOLD,HAMBURGER_PRICE,STORE,PERIOD,SOLD,HAMBURGER_PRICE
0,BLOOR_ST,1,27856,647,BROAD_ST,1,27856,593,COL_ST,1,...,27856,454,HIGH_ST,1,27856,402,ITAL_VIL,1,27856,384
1,BLOOR_ST,2,30356,643,BROAD_ST,2,30356,594,COL_ST,2,...,30356,456,HIGH_ST,2,30356,403,ITAL_VIL,2,30356,385
2,BLOOR_ST,3,32856,644,BROAD_ST,3,32856,590,COL_ST,3,...,32856,452,HIGH_ST,3,32856,405,ITAL_VIL,3,32856,382
3,BLOOR_ST,4,35356,646,BROAD_ST,4,35356,591,COL_ST,4,...,35356,453,HIGH_ST,4,35356,401,ITAL_VIL,4,35356,383
4,BLOOR_ST,5,37856,647,BROAD_ST,5,37856,593,COL_ST,5,...,37856,454,HIGH_ST,5,37856,402,ITAL_VIL,5,37856,384
5,BLOOR_ST,6,40356,643,BROAD_ST,6,40356,594,COL_ST,6,...,40356,456,HIGH_ST,6,40356,403,ITAL_VIL,6,40356,385
6,BLOOR_ST,7,42856,644,BROAD_ST,7,42856,590,COL_ST,7,...,42856,452,HIGH_ST,7,42856,405,ITAL_VIL,7,42856,382
7,BLOOR_ST,8,45356,646,BROAD_ST,8,45356,592,COL_ST,8,...,45356,453,HIGH_ST,8,45356,401,ITAL_VIL,8,45356,383
8,BLOOR_ST,9,47856,647,BROAD_ST,9,47856,593,COL_ST,9,...,47856,455,HIGH_ST,9,47856,402,ITAL_VIL,9,47856,384
9,BLOOR_ST,10,50356,643,BROAD_ST,10,50356,594,COL_ST,10,...,50356,456,HIGH_ST,10,50356,404,ITAL_VIL,10,50356,385
